In [3]:
from pathlib import Path
import pickle
import numpy as np
import pretty_midi

In [4]:
artists_songs_paths = {
    'Bach': ['data/orchestra/bach'],
    'Mozart': ['data/orchestra/mozart'],
    'Schumann x Chopin': ['data/orchestra/schumann', 'data/orchestra/chopin'],
    'Debussy x Albeniz': ['data/orchestra/debussy', 'data/orchestra/albeniz'],
    'All': ['data/orchestra']
}

artists_notes_paths = {
    'Bach': 'data/notes_files/bach_notes',
    'Mozart': 'data/notes_files/mozart_notes',
    'Schumann x Chopin': 'data/notes_files/schumann_chopin_notes',
    'Debussy x Albeniz': 'data/notes_files/debussy_albeniz_notes',
    'All': 'data/notes_files/all_notes'
}

In [5]:
artist_notes = {}

In [6]:
for artist, paths in artists_songs_paths.items():
    
    training_notes = []
    
    # if artists_notes_paths[artist] file already exists, skip this artist
    if Path(artists_notes_paths[artist]).exists():
        with open(artists_notes_paths[artist], 'rb') as filepath:
            training_notes = pickle.load(filepath)
        
        artist_notes[artist] = training_notes
        
        continue

    for path in paths:
        for count, midi_file in enumerate(Path(path).rglob('*.mid')):
                print(f"Parsing {midi_file}")
                notes = []
                durations = []

                try:
                    #parsing MIDI files one by one
                    midi_data = pretty_midi.PrettyMIDI(str(midi_file))
                except:
                    print("Error parsing file: {}".format(midi_file))
                    continue

                #depending on the element found in the instrument. Like we would have 'rest' for drums,
                #chords for guitar
                instrument = midi_data.instruments[0] if len(midi_data.instruments) > 0 else None
                
                if instrument is None:
                    continue
                
                sorted_notes = sorted(instrument.notes, key=lambda x: x.start)
                
                chord = [instrument.notes[0]]
                chord_end = instrument.notes[0].end
                first = True
                for note in sorted_notes:
                    if first:
                        first = False
                        continue
                    
                    if note.start > chord_end:
                        notes.append(chord)
                        chord = []
                        chord_end = note.end
                        chord.append(note)
                    else:
                        chord.append(note)
                        chord_end = note.end if note.end > chord_end else chord_end
                
                #notes and duration hold seuence for one music piece. 
                training_notes.append(notes)
    
    artist_notes[artist] = training_notes

    with open(artists_notes_paths[artist], 'wb') as filepath:
        pickle.dump(training_notes, filepath)


In [7]:
class Notes:
    def __init__(self, notes):
        self.notes = notes
        notes = sorted(notes, key=lambda x: x.start)

        # first note is the one with the earliest start time
        self.first_note = notes[0]
        self.start = self.first_note.start

        notes = sorted(notes, key=lambda x: x.end)
        #last note is the one with the latest end time
        self.last_note = notes[-1]
        self.end = self.last_note.end
        self.duration = self.end - self.start
        self.parse_notes()
        
    def parse_notes(self):
        for note in self.notes:
            note.start -= self.start
            note.end -= self.start
            
    def __repr__(self) -> str:
        return f"Notes: len={len(self.notes)}, start={self.start}, end={self.end}, duration={self.duration}"
    
    def get_last_notes(self, n):
        if len(self.notes) < n:
            return self.notes
        return self.notes[-n:]

In [8]:
artist_notes = {key: [[Notes(notes) for notes in song] for song in training_notes] for key, training_notes in artist_notes.items()}

In [9]:
artist_notes = {key:[item for sublist in training_notes for item in sublist] for key, training_notes in artist_notes.items()}


In [10]:
N_PREVIOUS_NOTES = 5

In [11]:
def make_previous_tuple(notes, n=2):
    for i in range(len(notes) - n + 1):
        lst = []
        for j in range(n):
            lst.append((notes[i+j], i + j))
        yield lst

    
previous_artist_notes = {key:make_previous_tuple(training_notes, N_PREVIOUS_NOTES) for key, training_notes in artist_notes.items()}

In [12]:
artist_note_dicts = {}

for artist, previous_notes in previous_artist_notes.items():
    
    note_dict = {}

    for note_idx_pairs in previous_notes:
        for idx, (note, note_idx) in enumerate(note_idx_pairs):
            if idx == 0:
                continue
            
            for n in range(1, N_PREVIOUS_NOTES):
                if sum([len(note.notes) for note, _ in note_idx_pairs[:idx]]) < n:
                    continue
                
                previous_notes = [note.notes for note, _ in note_idx_pairs[:idx]]
                flattened_notes = [item for sublist in previous_notes for item in sublist][-n:]
                key = tuple([note.pitch for note in flattened_notes])
                if key in note_dict.keys():
                    if (note, note_idx) not in note_dict[key]:
                        note_dict[key].append((note, note_idx))
                else:
                    note_dict[key] = [(note, note_idx)]
            
    artist_note_dicts[artist] = note_dict

In [13]:
n_words = 500

In [24]:
chains = {}

for key in artist_notes.keys():
    random_idx = np.random.randint(0, len(artist_notes[key]))
    chains[key] =  [(artist_notes[key][random_idx], random_idx)]


In [22]:
artist_note_dicts['Bach'].keys()

dict_keys([(76,), (72, 76), (67, 72, 76), (76, 67, 72, 76), (77,), (74, 77), (69, 74, 77), (76, 69, 74, 77), (77, 69, 74, 77), (67, 74, 77), (77, 67, 74, 77), (77, 67, 72, 76), (81,), (76, 81), (69, 76, 81), (76, 69, 76, 81), (81, 69, 76, 81), (74,), (69, 74), (66, 69, 74), (81, 66, 69, 74), (74, 66, 69, 74), (79,), (74, 79), (67, 74, 79), (74, 67, 74, 79), (79, 67, 74, 79), (72,), (67, 72), (64, 67, 72), (79, 64, 67, 72), (72, 64, 67, 72), (66, 72), (62, 66, 72), (72, 62, 66, 72), (71,), (67, 71), (62, 67, 71), (72, 62, 67, 71), (71, 62, 67, 71), (73,), (67, 73), (64, 67, 73), (71, 64, 67, 73), (73, 64, 67, 73), (62, 69, 74), (73, 62, 69, 74), (74, 62, 69, 74), (65, 71), (62, 65, 71), (74, 62, 65, 71), (71, 62, 65, 71), (60, 67, 72), (71, 60, 67, 72), (72, 60, 67, 72), (65,), (60, 65), (57, 60, 65), (72, 57, 60, 65), (65, 57, 60, 65), (59, 65), (55, 59, 65), (65, 55, 59, 65), (64,), (60, 64), (55, 60, 64), (65, 55, 60, 64), (64, 55, 60, 64), (58, 60, 64), (64, 58, 60, 64), (57, 60, 64

In [26]:
def get_last_notes(notes, n):
    all_notes = [note.notes for note in notes]
    flattened_notes = [item for sublist in all_notes for item in sublist]
    return flattened_notes[-n:]

In [28]:
complete_chains = {}
for artist, chain in chains.items():
    complete_chain = chain.copy()
    for i in range(n_words):
        notes_1, idx = complete_chain[-1]
        last_notes = get_last_notes([note for note, _ in complete_chain], N_PREVIOUS_NOTES - 1)

        possible_keys = list(set([tuple([note.pitch for note in last_notes[-n:]]) for n in range(1, N_PREVIOUS_NOTES)]))
        possible_keys = [key for key in possible_keys if key in artist_note_dicts[artist].keys()]

        key_probability_distribs = [len(key) for key in possible_keys]
        key_probability_distribs = np.array(key_probability_distribs) / sum(key_probability_distribs)
        
        prob = np.random.rand()
        
        if prob < 0.5 or len(possible_keys) == 0:
            if idx + 1 >= len(artist_notes[artist]):
                complete_chain.append((Notes([pretty_midi.Note(0, 0, 0, 1)]), -1))
                continue
            new_note = artist_notes[artist][idx + 1]
            complete_chain.append((new_note, idx + 1))
            continue
        
        selected_key = np.random.choice(possible_keys, p=key_probability_distribs)
        selected = artist_note_dicts[artist][selected_key]
        idx = np.random.choice(len(selected))
        complete_chain.append(selected[idx])
    
    complete_chains[artist] = complete_chain


C:\Users\nrtc\AppData\Local\Temp\ipykernel_21224\1606372220.py:24: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  selected_key = np.random.choice(possible_keys, p=key_probability_distribs)


In [29]:
complete_chains['Bach']

[(Notes: len=3, start=10.542302737499996, end=11.141121487499996, duration=0.5988187499999995,
  13),
 (Notes: len=3, start=11.535858487499997, end=12.10363028958333, duration=0.5677718020833336,
  14),
 (Notes: len=3, start=12.122903237499996, end=12.71505318958333, duration=0.592149952083334,
  15),
 (Notes: len=3, start=13.12045868958333, end=13.693520289583331, duration=0.5730616000000008,
  16),
 (Notes: len=3, start=15.297512487499997, end=15.891011737499996, duration=0.5934992499999989,
  19),
 (Notes: len=3, start=14.70647868958333, end=15.278250289583331, duration=0.5717716000000017,
  18),
 (Notes: len=3, start=15.297512487499997, end=15.891011737499996, duration=0.5934992499999989,
  19),
 (Notes: len=3, start=16.283168737499995, end=16.847258581249996, duration=0.5640898437500006,
  20),
 (Notes: len=3, start=16.86639787708333, end=17.45086810208333, duration=0.5844702250000005,
  21),
 (Notes: len=3, start=17.85355260208333, end=18.43150735208333, duration=0.57795475,
  22

In [30]:
instrument_name = 'Orchestral Harp'

In [31]:
for artist, chain in complete_chains.items():
    offset = 0
    # Create a PrettyMIDI object
    chord = pretty_midi.PrettyMIDI()
    program = pretty_midi.instrument_name_to_program(instrument_name)
    instrument = pretty_midi.Instrument(program=program)

    for notes_, _ in chain:
        for new_note in notes_.notes:
            note = pretty_midi.Note(
                velocity=new_note.velocity, pitch=new_note.pitch, start=offset+new_note.start, end=offset+new_note.end)
            instrument.notes.append(note)

        offset += notes_.duration

    chord.instruments.append(instrument)
    # Write out the MIDI data
    chord.write(f"results/test_{artist}_{instrument_name}.mid")